In [1]:
# SETUP

import pandas as pd
import os
from copy import deepcopy
import numpy as np
from math import isclose

os.chdir("..")
from adaptive_multigrid import give_data_individual_names, find_best_individual_name, build_individual_dict_from_param_grid, update_param_grid
from psuu import scenario_configs, build_next_param_config_code
from model.config.params import *



sweep = "gateway_viability_sweep_ag1_"
sweep_family = sweep[:sweep.index("_ag")+3]
kpis = pd.read_csv("simulation_data/{}.csv".format(sweep), index_col=0)

exec("param_config = {}".format(sweep))
next_name = sweep[:-2] + str(int(sweep[-2]) + 1) + "_"
fitness_func = scenario_configs[sweep_family]['fitness_function']
variable_params = scenario_configs[sweep_family]['variable_params']
control_params = scenario_configs[sweep_family]['control_params']

/Users/seanmcowen/opt/anaconda3/envs/BlockScience/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
thresholds = {"a1": .1,
             "a2": .9,
             "s1": 750*1e6,
             "s2": .9,
             "t1": 7500*1e6,
             "t2": .9}
grouping = kpis.groupby(["param_" + x for x in variable_params])

# Get average cost
thresh1 = grouping.apply(lambda x: x["KPI 14"].mean())
# Check if bounded between
thresh1 = (thresh1 > thresholds["a1"]) & (thresh1 < thresholds["a2"])

# Check average monte carlo success for servicer NPV
thresh2 = grouping.apply(lambda x: (x["KPI 1"] > thresholds["s1"]).mean())
# Check number of successful parameter constellations
thresh2 = thresh2 > thresholds["s2"]

# Check average monte carlo success for gateway NPV
thresh3 = grouping.apply(lambda x: (x["KPI 3"] > thresholds["t1"]).mean())
# Check number of successful parameter constellations
thresh3 = thresh3 > thresholds["t2"]

thresh = pd.concat([thresh1, thresh2, thresh3], axis=1)
thresh.columns = ["Threshold Servicer Capital Costs", "Threshold Servicer NPV", "Threshold Gateway NPV"]
print(thresh.mean())

Threshold Servicer Capital Costs    0.0
Threshold Servicer NPV              0.0
Threshold Gateway NPV               0.0
dtype: float64


In [6]:
kpis[["KPI 1", "KPI 3", "KPI 14"]].describe()

,KPI 1,KPI 3,KPI 14
count,1.440000e+03,1440.000000,1440.000000
mean,-8.497433e+09,-96529.034382,40.446563
std,5.774389e+09,96262.619642,57.945847
min,-1.471072e+10,-258517.656254,3.347403
25%,-1.398806e+10,-185000.000000,10.164852
50%,-1.074065e+10,-87719.995060,22.225275
75%,-1.492377e+09,-1428.648016,53.600293
max,3.179284e+09,-986.474193,1297.367310
